In [71]:
import random
import numpy as np

from datetime import timedelta
import math
from utilities import generate_random_date, generate_licence_plate

In [72]:
car_brands = {
  "Bmw": ["Series 3"],
  "Toyota": ["Corolla", "Land Cruiser", "Hilux", "Rav"],
  "Skoda": ["Superb"],
  "Kia": ["Stinger GT", "Ceed 3"],
  "Volkswagen": ["Crafter", "Passat"],
  "Hyundai": ["Ionia", "Tucson"]
}

motorbike_brands = {
  "Triumph": ["Tiger 1050"],
  "Kawasaki": ["Versys 1000"],
  "BMW": ["R 1250 RT"],
}

In [73]:
from numpy import loadtxt
names_male = loadtxt("./data/supporting/names/polish_male_firstnames.txt", dtype='str')
names_female = loadtxt("./data/supporting/names/polish_female_firstnames.txt", dtype='str')
surnames = loadtxt("./data/supporting/names/polish_surnames.txt", dtype='str')
genders = ["female", "male"]
ranks = ["officer", "normal"]

print(len(names_male))
print(len(names_female))
print(len(surnames))

300
300
314549


# Define object classes

In [74]:
class Position:
  def __init__(self, lat, lng):
    
    self.lat = lat
    self.lng = lng

In [75]:
class Officer:
  def __init__(self, officer_id, first_name, last_name, rank, gender, date_of_birth, hire_date, email, phone_number):
    
    self.officer_id = officer_id
    self.first_name = first_name
    self.last_name = last_name
    self.rank = rank
    self.gender = gender
    self.date_of_birth = date_of_birth
    self.hire_date = hire_date
    self.email = email
    self.phone_number = phone_number

In [76]:
class Vehicle:
  def __init__(self, license_plate_number, vehicle_type, brand, model, manufacturing_year, position):
    
    self.license_plate_number = license_plate_number
    
    # "car" or "motorbike"
    self.vehicle_type = vehicle_type
    self.brand = brand
    self.model = model
    self.manufacturing_year = manufacturing_year
    self.position = position
    
    # meters per second
    # self.max_speed = 100
    self.last_inspection = manufacturing_year
    
    self.chased_incident = None
    self.is_resolving_incident = False
    
    self.officers = list()

In [77]:
class Incident:
  def __init__(self, incident_id, city, district, report_datetime, description, position):
    
    self.incident_id = incident_id
    self.city = city
    self.district = district
    self.report_datetime = report_datetime
    self.arrival_datetime = None
    self.victim_satisfaction = None
    self.description = description
    self.position = position
    
    self.victims = list()

In [78]:
class Victim:
  def __init__(self, victim_id, first_name, last_name, gender, date_of_birth):
    
    self.victim_id = victim_id
    self.first_name = first_name
    self.last_name = last_name
    self.gender = gender
    self.date_of_birth = date_of_birth

In [79]:
class City:
  def __init__(self, city_name, city_districts):
    
    self.city_name = city_name
    self.city_districts = city_districts

# Define Generator

In [80]:
def calculate_distance(pos_1, pos_2):
  return math.sqrt((pos_1.lat - pos_2.lat)**2 + (pos_1.lng - pos_2.lng)**2)

In [81]:
# Both ends included
def random_range(start, end, INTERNAL_SUBSTEPS=1000):
  INTERNAL_SUBSTEPS = 1000
  return random.randrange(start*INTERNAL_SUBSTEPS, end*INTERNAL_SUBSTEPS+1, 1) / INTERNAL_SUBSTEPS

In [82]:
def generate_personal_info():
  gender = random.choice(genders)
  surname = random.choice(surnames)
  first_name = ""
  
  if gender == "male":
    first_name = random.choice(names_male)
  else:
    first_name = random.choice(names_female)
    
  return {
    "gender": gender,
    "first_name": first_name,
    "surname": surname
  }  

In [83]:
INITIAL_COUNTS = {
  "officers": 100,
  "cars": 70,
  "motorbikes": 30
}

MERCATOR_PER_METER = 2.245789145352464e-6

# 30 meters
RANDOM_POS_STEP = MERCATOR_PER_METER * 30


class Generator:
  
  def __init__(self, initial_counts, initial_date, simulation_timestep, incidents_per_hour):
    
    self.initial_counts = initial_counts
    
    # In seconds
    self.simulation_timestep = simulation_timestep
    self.incidents_per_hour = incidents_per_hour
    
    
    self.vehicles = list()
    self.officers = list()
    
    self.incidents = list()
    self.victims = list()
    
    self.current_time = initial_date
  
  
  def create_officer(self, id):
    MIN_OFFICER_AGE = 21
    MAX_OFFICER_AGE = 70
    MAX_OFFICER_HIRE_TIME = 35
    
    personal_info = generate_personal_info()

    hire_date = generate_random_date(self.current_time - timedelta(days=MAX_OFFICER_HIRE_TIME*365), self.current_time)
    
    date_of_birth = generate_random_date(self.current_time - timedelta(days=MAX_OFFICER_AGE*365), hire_date - timedelta(days=MIN_OFFICER_AGE*365))
    
    rank = random.choice(ranks)

    
    officer = Officer(id, personal_info["first_name"], personal_info["surname"], rank, personal_info["gender"], date_of_birth, hire_date, "officer{}@police.org.pl".format(id), 111111111)
    # print(date_of_birth, hire_date)
    
    return officer
  
  def create_car(self):
    
    OLDEST_CAR_YEAR = 10
    EARLIEST_CAR_YEAR = 1
    
    license_plate_number = generate_licence_plate()
    brand = random.choice(list(car_brands.keys()))
    model = random.choice(car_brands[brand])
    manufacturing_year = generate_random_date(self.current_time - timedelta(days=OLDEST_CAR_YEAR*365), self.current_time - timedelta(days=EARLIEST_CAR_YEAR*365))
    
    # FIX it - just an testing position
    position = Position(45, 45)
    car = Vehicle(license_plate_number, "car", brand, model, manufacturing_year, position)
    # print(manufacturing_year, brand, model, license_plate_number)
    
    return car
  
  def create_motorbike(self):
    OLDEST_CAR_YEAR = 10
    EARLIEST_CAR_YEAR = 1
    
    license_plate_number = generate_licence_plate()
    brand = random.choice(list(motorbike_brands.keys()))
    model = random.choice(motorbike_brands[brand])
    manufacturing_year = generate_random_date(self.current_time - timedelta(days=OLDEST_CAR_YEAR*365), self.current_time - timedelta(days=EARLIEST_CAR_YEAR*365))
    
    # FIX it - just an testing position
    position = Position(45, 45)
    motorbike = Vehicle(license_plate_number, "motorbike", brand, model, manufacturing_year, position)
    # print(manufacturing_year, brand, model, license_plate_number)
    
    return motorbike
  
  def generate_initial_data(self):
    
    for i in range(0, self.initial_counts["officers"]):
      officer = self.create_officer(i)
      self.officers.append(officer)
      
    for i in range(0, self.initial_counts["cars"]):
      car = self.create_car()
      self.vehicles.append(car)
      
    for i in range(0, self.initial_counts["motorbikes"]):
      motorbike = self.create_motorbike()
      self.vehicles.append(motorbike)
  
  
  def move_vehicle(self, vehicle):
    
    # In m/s
    DEFAULT_VEHICLE_SPEED = 14
    # EMERGENCY_CAR_SPEED = 25
    # EMERGENCY_MOTORBIKE_SPEED = 30

    DELTA_MERCATOR_MAIN = self.simulation_timestep * DEFAULT_VEHICLE_SPEED * MERCATOR_PER_METER

    if vehicle.chased_incident == None and not vehicle.is_resolving_incident:
      delta_lat = 0
      delta_lng = 0
      
      axis = random.choice([True, False])
      direction = random.choice([-1, 1])
      
      distance = random.randrange(int(DELTA_MERCATOR_MAIN / RANDOM_POS_STEP * 0.75), int(DELTA_MERCATOR_MAIN / RANDOM_POS_STEP * 1.25)+1, 1) * RANDOM_POS_STEP
      
      # print(DELTA_MERCATOR_MAIN)
      if axis:
        delta_lat = distance * direction
      else:
        delta_lng = distance * direction
        
      vehicle.position.lat += delta_lat
      vehicle.position.lng += delta_lng
    elif vehicle.chased_incident != None and not vehicle.is_resolving_incident:
      
      
      d_lat = vehicle.chased_incident.position.lat - vehicle.position.lat
      d_lng = vehicle.chased_incident.position.lng - vehicle.position.lng
      dist_to_incident = math.sqrt((d_lat)**2 + (d_lng)**2)
      
      dir_lat = (d_lat / dist_to_incident) * (d_lat / abs(d_lat))
      dir_lng = (d_lng / dist_to_incident) * (d_lat / abs(d_lat))
      
      step_distance = random.randrange(int(DELTA_MERCATOR_MAIN / RANDOM_POS_STEP * 0.75), int(DELTA_MERCATOR_MAIN / RANDOM_POS_STEP * 1.25)+1, 1) * RANDOM_POS_STEP
      
      # print("Should go directly toward incident", dir_lat, dir_lng, dist_to_incident, step_distance)
      
      if step_distance > dist_to_incident:
        print("Stop and handle incident")
        
        vehicle.position.lat = vehicle.chased_incident.position.lat
        vehicle.position.lng = vehicle.chased_incident.position.lng
        
        # Calculate arrival time depending on travelled distance to distance to incident proportions
        distances_ratio = dist_to_incident / step_distance
        
        vehicle.chased_incident.arrival_datetime = self.current_time + timedelta(seconds=self.simulation_timestep * distances_ratio)
        
        
        vehicle.chased_incident = None
        vehicle.is_resolving_incident = True
      else:
        # step_distance_lat = dir_lat * step_distance
        # step_distance_lng = dir_lng * step_distance
        vehicle.position.lat += dir_lat * step_distance
        vehicle.position.lng += dir_lng * step_distance
        
        # print("Move by: ", dir_lat * step_distance, dir_lng * step_distance)
      
         
  def move_vehicles(self):
    
    for vehicle in self.vehicles:
      self.move_vehicle(vehicle)    
    
  def generate_victims_for_incident(self, incident):
    victims_count = np.random.poisson(lam=0.4, size=1)[0]+1
    
    # print(victims_count)
    date_of_birth = generate_random_date(self.current_time - timedelta(days=90*365), self.current_time - timedelta(days=2*365))
    
    for i in range(victims_count):
      
      personal_info = generate_personal_info()
    
      victim = Victim(len(self.victims), personal_info["first_name"], personal_info["surname"], personal_info["gender"], date_of_birth)
      self.victims.append(victim)
      incident.victims.append(victim)
      
  def assign_vehicle_to_incident(self, incident):   
    closest_vehicle = None #self.vehicles[0]
    closest_dist = 0 # calculate_distance(incident.position, closest_vehicle.position)
    
    for vehicle in self.vehicles:
      
      if vehicle.chased_incident == None and vehicle.is_resolving_incident == False:
        distance = calculate_distance(incident.position, vehicle.position)
        
        if closest_vehicle == None or distance < closest_dist:
          closest_dist = distance
          closest_vehicle = vehicle
        # elif distance < closest_dist:
        #   closest_dist = distance
        #   closest_vehicle = vehicle
        
    # We know the closest vehicle to the incident, now we assign it
    
    if closest_vehicle != None:
      closest_vehicle.chased_incident = incident
    else:
      print("All vehicles are currently occupied")
  
  def generate_incidents(self):
    avarage_incidents = self.simulation_timestep / 3600 * self.incidents_per_hour
    # Here, higher random range of 50%
    incidents_count = int(random_range(int(avarage_incidents * 0.5), int(avarage_incidents * 1.5), 1))
    # print(incidents_count)
    
    for i in range(incidents_count):
      
      city_lat_min = 44.7
      city_lat_max = 45.3
      city_lng_min = 44.7
      city_lng_max = 45.3
      
      position = Position(random_range(city_lat_min, city_lat_max, 100), random_range(city_lng_min, city_lng_max, 100000))
      
      incident_datetime = self.current_time + timedelta(seconds=self.simulation_timestep * random_range(0, 1, 1000))
      # TODO Should add victims to incident
      
      incident = Incident(len(self.incidents), "city", "district", incident_datetime, "some description", position)
      self.generate_victims_for_incident(incident)
      
      # TODO Should assign car team to incident and add victims
      self.assign_vehicle_to_incident(incident)
      
      self.incidents.append(incident)
    
  def simulate(self, start_datetime, end_datetime):
    time_diff = end_datetime - start_datetime
    iterations = time_diff.total_seconds() / self.simulation_timestep
    
    timestep = timedelta(seconds=self.simulation_timestep)
    # print(time_diff.total_seconds(), iterations)

    for i in range(0, int(iterations)):
      self.generate_incidents()
      self.move_vehicles()
      
      # print(self.cars[0].position.lat, self.cars[0].position.lng)
      # print(i, str(current_time))
      
      self.current_time = start_datetime + i * timestep

In [84]:
from datetime import datetime
startdate = datetime(2023, 1, 1)
enddate = datetime(2023, 1, 31)

generator = Generator(INITIAL_COUNTS, startdate, 600, 12)

generator.generate_initial_data()

test_incident = Incident(0, "city", "district", startdate, "some description", Position(45.05, 45.1))
generator.vehicles[0].chased_incident = test_incident
# print()

generator.simulate(startdate, enddate)

Stop and handle incident
Stop and handle incident
Stop and handle incident
Stop and handle incident
Stop and handle incident
Stop and handle incident
Stop and handle incident
Stop and handle incident
Stop and handle incident
Stop and handle incident
Stop and handle incident
Stop and handle incident
Stop and handle incident
Stop and handle incident
Stop and handle incident
Stop and handle incident
Stop and handle incident
Stop and handle incident
Stop and handle incident
Stop and handle incident
Stop and handle incident
Stop and handle incident
Stop and handle incident
Stop and handle incident
Stop and handle incident
Stop and handle incident
Stop and handle incident
Stop and handle incident
Stop and handle incident
Stop and handle incident
Stop and handle incident
Stop and handle incident
Stop and handle incident
Stop and handle incident
Stop and handle incident
Stop and handle incident
Stop and handle incident
Stop and handle incident
Stop and handle incident
Stop and handle incident


C:\Users\Pawel\AppData\Local\Temp\ipykernel_7200\1746000381.py:4: DeprecationWarning: non-integer arguments to randrange() have been deprecated since Python 3.10 and will be removed in a subsequent version
  return random.randrange(start*INTERNAL_SUBSTEPS, end*INTERNAL_SUBSTEPS+1, 1) / INTERNAL_SUBSTEPS


All vehicles are currently occupied
All vehicles are currently occupied
All vehicles are currently occupied
All vehicles are currently occupied
All vehicles are currently occupied
All vehicles are currently occupied
All vehicles are currently occupied
All vehicles are currently occupied
All vehicles are currently occupied
All vehicles are currently occupied
All vehicles are currently occupied
All vehicles are currently occupied
All vehicles are currently occupied
All vehicles are currently occupied
All vehicles are currently occupied
All vehicles are currently occupied
All vehicles are currently occupied
All vehicles are currently occupied
All vehicles are currently occupied
All vehicles are currently occupied
All vehicles are currently occupied
All vehicles are currently occupied
All vehicles are currently occupied
All vehicles are currently occupied
All vehicles are currently occupied
All vehicles are currently occupied
All vehicles are currently occupied
All vehicles are currently o

In [85]:
print(len(generator.incidents))

6477
